In [1]:
"""Main Script to Scrape and process all the news articles in real time and store them Supabase
Should be ran every 24hours?
"""
# Author: Kaleb-Nim
# Date: 11th September 2023 
from utils.Webscraper import Webscraper
from Supabase.Extractor import SupabaseExtractor
from Supabase.Insertor import SupabaseInsertor
from llm.mde import MDE
import logging
import os
from time import time
logger = logging.getLogger(__name__)

In [2]:
# Load all news sources we want to webscrape from Supabase Source Table
baseSources  = SupabaseExtractor.extractSources()
# Get the base_url for each source
base_urls = [source['domain_url'] for source in baseSources]
sample_base_urls = [base_urls[1]]
# Build all the source objects, takes ~1.5min per source
source_objects = Webscraper.downloadAllSources(base_urls=sample_base_urls)

2023-09-12 10:32:57,716:INFO - HTTP Request: GET https://exeakzvxpuwdfeavnjgn.supabase.co/rest/v1/Source?select=%2A "HTTP/1.1 200 OK"
2023-09-12 10:32:57,719:INFO - Number of base news sources in Supabase: 1
2023-09-12 10:32:59,395:CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: https://www.scmp.com/feeds
2023-09-12 10:33:43,676:INFO - Time taken to build all source objects: 45.95655417442322 seconds
2023-09-12 10:33:43,677:INFO - Number of articles scraped from https://www.scmp.com/: 190


In [3]:
all_url_list = []
for source in source_objects:
    for article in source.articles:
        all_url_list.append(article.url)
print(f'Number of articles scraped: {len(all_url_list)}')


Number of articles scraped: 190


In [4]:
# Extract all the Supabase urls
supabase_urls = SupabaseExtractor.extractAllArticleUrl()
supabase_urls

2023-09-12 10:33:44,142:INFO - HTTP Request: GET https://exeakzvxpuwdfeavnjgn.supabase.co/rest/v1/Article?select=Url "HTTP/1.1 200 OK"


['https://www.straitstimes.com/life/travel/on-the-road-mountain-hopping-in-northern-thailand',
 'https://www.straitstimes.com/asia/australianz/56-magnitude-quake-hits-central-new-zealand',
 'https://www.usnews.com/news/us/articles/2023-08-30/idalia-predicted-to-hit-florida-as-category-4-hurricane-with-catastrophic-storm-surge']

In [5]:
# Compare the two lists and remove any duplicates
new_urls = Webscraper._extract_non_duplicates_urls(all_url_list, supabase_urls)
print(f'Number of new articles: {len(new_urls)}')

Number of new articles: 190


In [6]:
# Remove articles that are duplicates from the source
new_articles = []
for source in source_objects:
    for article in source.articles:
        if article.url in new_urls:
            new_articles.append(article)
print(f'Number of new articles: {len(new_articles)}')

Number of new articles: 190


In [13]:
# Testing purposes only that's why 5
import random
random.shuffle(new_articles)

In [16]:
random_articles10 = new_articles[:10]
ten_articles = [article.url for article in random_articles10]
ten_articles

['https://www.scmp.com/video/world/3230851/first-us-picklemall-opens-public-breathing-new-life-dying-shopping-malls?module=visual_stories&pgtype=section',
 'https://www.scmp.com/comment/opinion/article/3233689/beyond-china-economies-us-and-europe-are-no-less-vulnerable',
 'https://www.scmp.com/economy/global-economy/article/3234140/vietnam-ups-durian-trade-china-land-border-offers-cost-advantage-challenge-thai-dominance',
 'https://www.scmp.com/tech/big-tech/article/3234147/chinese-ride-hailing-giant-didi-global-sees-second-quarter-revenue-rise-and-losses-narrow-strong',
 'https://www.scmp.com/business/companies/article/3234156/farming-flies-freeze-drying-ugly-fruit-and-jewellery-scraps-how-hong-kong-start-ups-are-reducing?module=live&pgtype=homepage',
 'https://www.scmp.com/comment/opinion/article/3230344/how-ais-rise-could-be-boon-manufacturing-and-healthcare-sectors-asia',
 'https://www.scmp.com/comment/opinion/article/3233610/china-must-slow-down-investment-if-it-wants-rebalance-it

## Run from here maybe?

In [17]:
ten_articles = ['https://www.reuters.com/business/environment/remnants-typhoon-haikui-cause-floods-southeastern-china-2023-09-06/',
 'https://www.scmp.com/comment/opinion/article/3233689/beyond-china-economies-us-and-europe-are-no-less-vulnerable',
 'https://www.scmp.com/economy/global-economy/article/3234140/vietnam-ups-durian-trade-china-land-border-offers-cost-advantage-challenge-thai-dominance',
 'https://www.straitstimes.com/world/race-to-find-survivors-as-morocco-quake-deaths-top-1300',
 'https://www.scmp.com/business/companies/article/3234156/farming-flies-freeze-drying-ugly-fruit-and-jewellery-scraps-how-hong-kong-start-ups-are-reducing?module=live&pgtype=homepage',
 'https://www.scmp.com/comment/opinion/article/3230344/how-ais-rise-could-be-boon-manufacturing-and-healthcare-sectors-asia',
 'https://www.scmp.com/comment/opinion/article/3233610/china-must-slow-down-investment-if-it-wants-rebalance-its-debt-laden-economy',
 'https://www.scmp.com/comment/opinion/article/3230626/us-china-tech-war-semiconductors-heart-competition-driving-world-towards-new-cold-war',
 'https://www.scmp.com/week-asia/politics/article/3234129/malaysian-pm-anwar-spend-billions-5-year-plan-reduce-poverty-boost-economic-growth',
 'https://www.scmp.com/comment/opinion/article/3234037/hong-kong-floods-even-climate-change-sceptics-must-see-need-prepare-extreme-weather?module=opinion&pgtype=homepage']

In [ ]:
ten_articles_nlped = Webscraper.parseNlpAllArticles(ten_articles)

In [9]:
# Process the articles
processed_articles = []
for article in random_articles10:
    processed_articles.append(MDE.process(article))

2023-09-12 10:33:50,318:ERROR - INFO: article:Anxiety, not ambition, behind rise of the Global South against US-led world order is not a disruption event article
2023-09-12 10:33:50,319:ERROR - Error: Binary Classification Failed -> 'text_body'
2023-09-12 10:33:52,998:ERROR - INFO: article:As West exploits India-China cracks, Asia must rise above the divisive ideology is not a disruption event article
2023-09-12 10:33:55,680:ERROR - INFO: article:Opinion | Marcos Jnr’s Philippine presidency an exercise in redemption is not a disruption event article
2023-09-12 10:33:58,589:ERROR - INFO: article:China’s proposed ban on ‘hurtful’ clothing is a worrying sign of advancing intolerance in society is not a disruption event article
2023-09-12 10:34:01,905:ERROR - INFO: article:Japan sex predator Johnny Kitagawa’s talent agency faces spiralling crisis amid growing anger, advertiser boycott is not a disruption event article
2023-09-12 10:34:01,908:ERROR - Error: Binary Classification Failed -> '

In [10]:
processed_articles[0].additional_data

AttributeError: 'str' object has no attribute 'additional_data'